In [1]:
from tools.google_trends import get_trends

keywords = get_trends('ko', 540, 'south_korea')
print(keywords)

['김여정', '이영지', '구성환', "엘든링' DLC", '임성근', '국민권익위원회 김건희', '얼차려 훈련병 사망사건 중대장 부중대장 구속', '스페인 이탈리아', '김도영', '쉬인', '맥도날드 감자튀김 판매 중단', '메이플스토리', '말라리아', '에오스블랙', '러브버그 해충', '태국 동성결혼', '엔비디아', '리그오브레전드', '코리아컵', '문재인']


In [2]:
import os
from dotenv import load_dotenv
from tools.news_crawler import NaverApiCrawler

load_dotenv()

client_id = os.getenv('NAVER_CLIENT_ID')
client_secret = os.getenv('NAVER_CLIENT_SECRET')
endpoint = os.getenv('NAVER_API_ENDPOINT')

crawler = NaverApiCrawler(client_id, client_secret, endpoint)

articles = {}
for keyword in keywords:
    articles[keyword] = crawler.get_articles(keyword, 100, 'date')

for keyword, article_list in articles.items():
    print('Keyword: %s \t\t Count: %d' % (keyword, len(article_list)))

Keyword: 김여정 		 Count: 100
Keyword: 이영지 		 Count: 100
Keyword: 구성환 		 Count: 100
Keyword: 엘든링' DLC 		 Count: 100
Keyword: 임성근 		 Count: 100
Keyword: 국민권익위원회 김건희 		 Count: 100
Keyword: 얼차려 훈련병 사망사건 중대장 부중대장 구속 		 Count: 100
Keyword: 스페인 이탈리아 		 Count: 100
Keyword: 김도영 		 Count: 100
Keyword: 쉬인 		 Count: 100
Keyword: 맥도날드 감자튀김 판매 중단 		 Count: 100
Keyword: 메이플스토리 		 Count: 100
Keyword: 말라리아 		 Count: 100
Keyword: 에오스블랙 		 Count: 100
Keyword: 러브버그 해충 		 Count: 100
Keyword: 태국 동성결혼 		 Count: 100
Keyword: 엔비디아 		 Count: 100
Keyword: 리그오브레전드 		 Count: 100
Keyword: 코리아컵 		 Count: 100
Keyword: 문재인 		 Count: 100


In [6]:
# Selenium으로 네이버 기사에서 제목, 내용 크롤링
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options().add_argument("--headless")


driver = webdriver.Chrome(options=options)

for keyword, items in tqdm(articles.items()):
    for item in items:
        driver.get(item['link'])
        WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, "_article_content")))
        title = driver.find_element(By.XPATH, "/html/head/title").get_attribute("innerText")
        description = " ".join([element.get_attribute("innerText") for element in driver.find_elements(By.CLASS_NAME, "_article_content")])
        item['title'] = title
        item['description'] = description

driver.close()

  0%|          | 0/20 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [8]:
# null check
title_null_cnt = 0
desc_null_cnt = 0

for keyword, items in articles.items():
    for item in items:
        if item['title'] == '':
            title_null_cnt += 1
        if item['description'] == '':
            desc_null_cnt += 1
print(title_null_cnt, desc_null_cnt)

0 0


In [9]:
for keyword, items in articles.items():
    for item in items:
        if item['description'] == '':
            print(item)

In [10]:
from datetime import datetime
import mysql.connector

MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PWD =  os.getenv('MYSQL_PWD')

conn = mysql.connector.connect(
    host = 'localhost',
    port = '3306',
    user = MYSQL_USER,
    password = MYSQL_PWD,
    charset = 'utf8mb4'
)

cursor = conn.cursor()

query = """
INSERT INTO news.articles (id, keyword, originallink, link, pubdate, title, description) VALUES (%s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE id = id;
"""

try:
    for keyword, items in articles.items():
        for item in items:
            if item['originallink'] == item['link']:
                continue
            id = ''.join(item['link'].split('/')[-2:])
            if '?' in id:
                id = id.split('?')[0]
            pubdatetime = datetime.strptime(item['pubDate'], '%a, %d %b %Y %H:%M:%S %z')
            pubdate_str = pubdatetime.strftime("%Y-%m-%d %H:%M:%S")
            data = (id, keyword, item['originallink'], item['link'], pubdate_str, item['title'], item['description'])
            cursor.execute(query, data)
    conn.commit()
except Exception as e:
    print(e)
finally:
    conn.close()



In [11]:
datetime.today().strftime('%Y-%m-%d')

'2024-06-19'

In [12]:
# MySQL에서 오늘자 기사 받아오기
from datetime import datetime
import mysql.connector
import pymysql
import pymysql.cursors

MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PWD =  os.getenv('MYSQL_PWD')

conn = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = MYSQL_USER,
    password = MYSQL_PWD,
    charset = 'utf8mb4'
)

cursor = conn.cursor(pymysql.cursors.DictCursor)

query = """
SELECT * from news.articles where DATE(date) = %s;
"""

try:
    today = datetime.today().strftime('%Y-%m-%d')
    data = (today,)
    cursor.execute(query, data)
    output = cursor.fetchall()
except Exception as e:
    print(e)
finally:
    conn.close()


In [13]:
print(len(output))
print(output[0])

934
{'id': 10007849751, 'keyword': '역전검사', 'date': datetime.datetime(2024, 6, 19, 22, 21, 48), 'originallink': 'http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.aspx?c=AKR20150911150700060&did=1195m', 'link': 'https://n.news.naver.com/mnews/article/001/0007849751?sid=102', 'pubdate': datetime.datetime(2015, 9, 11, 16, 32), 'title': '중고생 법정변론 대회…주장 서툴지만 열정은 법조인', 'description': '\n\n\n의정부지법 축제 프로그램…16개팀 참가해 열띤 공방\n\n(의정부=연합뉴스) 김도윤 기자 = "사적인 일로 사고 현장을 이탈해 뺑소니입니다." "필요한 조치를 한 뒤 돌아오겠다고 약속했으니 뺑소니가 아닙니다."\n\n11일 법 문화 축제인 \'청송제\'(聽訟祭)의 주요 프로그램 가운데 하나로 학생 법정변론 경연대회가 열린 의정부지법 5호 법정.\n\n이관용 부장판사를 비롯한 법관 3명의 심사로 진행된 의정부 충의중 \'역전검사\'팀과 효자중 \'효자\'팀의 첫 대결은 처음부터 팽팽했다.\n\n피고인 A씨가 승용차를 운전하다 옆을 지나가는 자전거를 쳐 B군의 무릎을 다치게 했으나 급한 일이 있다며 인근 편의점 주인에게 사고 처리를 부탁한 뒤 현장을 이탈한 가상 사건이 공방의 주제다.\n\nA씨는 몇 시간 뒤 사고 현장으로 돌아왔다가 자신에게 욕한 것으로 오해한 나머지 만취한 C씨를 때려 다치게도 해 뺑소니, 사고 후 미조치, 상해 등 3가지 혐의로 기소됐다.\n\n역전검사팀은 첫 변론에서 "피고인은 인적사항을 밝히지 않은데다 사고를 수습하지 않고 현장을 이탈했고 상해는 결과범인데 C씨가 더 많이 다쳤다"며 3가지 혐의 모두에 유죄를 주장했다.\n\n이에 효자팀은 "B군이 안전거리를 유지하지 

In [14]:
from opensearchpy import OpenSearch
from opensearch_dsl import Search

host = 'localhost'
port = 9200

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
)

In [15]:
# 기사 생성시점에 따라 각각 다른 데일리 인덱스에 저장


index_prefix = 'articles-'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 1
    }
  }
}

for obj in output:
    pubdate = obj['pubdate'].strftime('%Y.%m.%d')
    index_name = ''.join([index_prefix, pubdate])
    doc = {
        'keyword': obj['keyword'],
        'title': obj['title'],
        'description': obj['description'],
        'date': obj['date'].astimezone().strftime("%Y%m%dT%H%M%S%z"),
        'pubdate': obj['pubdate'].astimezone().strftime("%Y%m%dT%H%M%S%z"),
        'originallink': obj['originallink'],
        'link': obj['link']
    }
    if not client.indices.exists(index_name):
        client.indices.create(index_name, body=index_body)
    
    client.index(index = index_name, body = doc, id = obj['id'])
    
    



In [ ]:
# 기사 생성시점에 따라 각각 다른 데일리 인덱스에 저장


index_prefix = 'articles-'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 1
    }
  }
}

for obj in output:
    pubdate = obj['pubdate'].strftime('%Y.%m.%d')
    index_name = ''.join([index_prefix, pubdate])
    doc = {
        'keyword': obj['keyword'],
        'title': obj['title'],
        'description': obj['description'],
        'date': obj['date'].astimezone().strftime("%Y%m%dT%H%M%S%z"),
        'pubdate': obj['pubdate'].astimezone().strftime("%Y%m%dT%H%M%S%z"),
        'originallink': obj['originallink'],
        'link': obj['link']
    }
    if not client.indices.exists(index_name):
        client.indices.create(index_name, body=index_body)
    
    client.index(index = index_name, body = doc, id = obj['id'])
    
    



In [16]:
client.indices.exists("index=")

False

In [17]:
"yyyyMMddTHHmmssZ"
"20190323T213446-04:00"

'20190323T213446-04:00'

In [18]:
output[0]['pubdate'].strftime("'%Y%m%dT%H%M%Z")

"'20150911T1632"

In [19]:
from datetime import timezone
output[0]['pubdate'].astimezone().strftime("%Y%m%dT%H%M%S%z")


'20150911T163200+0900'